In [1]:
import torch,torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
PROJECT_NAME = 'Car-Object-Detection-V3-Learning-Detectron2-V2'

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


In [4]:
xmin,ymin,xmax,ymax = 281,187,327,223
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin

In [5]:
img = cv2.imread('./data/vid_4_1000.jpg')
img_crop = img[y:y+h,x:x+w]
cv2.imwrite('./crop.png',img_crop)
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite('./rectangle.png',img)

True

In [6]:
def load_data():
    new_data = []
    iters = len(data)
    for i in tqdm(range(iters)):
        info = data.iloc[i]
        record = {}
        height,width = cv2.imread(f"./data/{info['image']}").shape[:2]
        record['file_name'] = f"./data/{info['image']}"
        record['height'] = height
        record['width'] = width
        record['class_id'] = i
        record['image_id'] = i
        record['cateogry_id'] = 0
        record['annotations'] = [
            {
                "bbox":[info['xmin'],info['ymin'],info['xmax'],info['ymax']],
                "bbox_mode":BoxMode.XYXY_ABS,
                'category_id':0
            }
        ]
        new_data.append(record)
    return new_data

In [7]:
labels = ['car']

In [8]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=labels)
metadata = MetadataCatalog.get('data')

In [9]:
len(labels)

1

In [10]:
wandb.init(project=PROJECT_NAME,name='baseline',sync_tensorboard=True)
model = "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.DATASETS.TRAIN = ('data')
cfg.DATASETS.TEST = ()
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.SOLVER.STEPS = []
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
cfg.MODEL.WEIGHTS = "./output/model_final.pth"
cfg.SOLVER.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator('data',output_dir="./output/")
val_loader = build_detection_test_loader(cfg,'data')
wandb.log(inference_on_dataset(predictor.model,val_loader,evaluator))
img = cv2.imread('./data/vid_4_1000.jpg')
preds = predictor(img)
v = Visualizer(img[:,:,::-1],metadata=metadata)
v = v.draw_instance_predictions(preds['instances'].to('cpu'))
v = v.get_image()[:,:,::-1]
plt.figure(figsize=(12,6))
plt.imshow(v)
plt.savefig('./img.png')
plt.close()

2021-10-13 22:46:18.290528: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20211013_224619-3lw9jfzz
wandb: Syncing run baseline



[10/13 22:46:27 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)


100%|████████████████████████████████████████| 559/559 [00:01<00:00, 356.31it/s]

[10/13 22:46:29 d2.data.build]: Removed 0 images with no usable annotations. 559 images left.
[10/13 22:46:29 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    car     | 559          |
|            |              |
[10/13 22:46:29 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[10/13 22:46:29 d2.data.build]: Using training sampler TrainingSampler
[10/13 22:46:29 d2.data.common]: Serializing 559 elements to byte tensors and concatenating them all ...
[10/13 22:46:29 d2.data.common]: Serialized dataset takes 0.21 MiB



Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (2, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, w

[10/13 22:46:29 d2.engine.train_loop]: Starting training from iteration 0
[10/13 22:46:39 d2.utils.events]:  eta: 0:07:58  iter: 19  total_loss: 1.09  loss_cls: 0.623  loss_box_reg: 0.4767  loss_rpn_cls: 0.002588  loss_rpn_loc: 0.004985  time: 0.4807  data_time: 0.0143  lr: 4.9953e-06  max_mem: 3163M
[10/13 22:46:49 d2.utils.events]:  eta: 0:07:50  iter: 39  total_loss: 1.008  loss_cls: 0.5745  loss_box_reg: 0.4406  loss_rpn_cls: 0.003634  loss_rpn_loc: 0.006043  time: 0.4871  data_time: 0.0025  lr: 9.9902e-06  max_mem: 3163M
[10/13 22:46:59 d2.utils.events]:  eta: 0:07:42  iter: 59  total_loss: 0.9369  loss_cls: 0.4527  loss_box_reg: 0.4781  loss_rpn_cls: 0.003208  loss_rpn_loc: 0.003605  time: 0.4921  data_time: 0.0024  lr: 1.4985e-05  max_mem: 3163M
[10/13 22:47:09 d2.utils.events]:  eta: 0:07:32  iter: 79  total_loss: 0.8467  loss_cls: 0.3378  loss_box_reg: 0.4793  loss_rpn_cls: 0.002734  loss_rpn_loc: 0.005439  time: 0.4893  data_time: 0.0025  lr: 1.998e-05  max_mem: 3163M
[10/13 

100%|████████████████████████████████████████| 559/559 [00:01<00:00, 348.95it/s]

[10/13 22:54:56 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/13 22:54:56 d2.data.common]: Serializing 559 elements to byte tensors and concatenating them all ...
[10/13 22:54:56 d2.data.common]: Serialized dataset takes 0.21 MiB
[10/13 22:54:56 d2.evaluation.evaluator]: Start inference on 559 batches


[10/13 22:54:59 d2.evaluation.evaluator]: Inference done 11/559. Dataloading: 0.0006 s/iter. Inference: 0.2721 s/iter. Eval: 0.0001 s/iter. Total: 0.2728 s/iter. ETA=0:02:29
[10/13 22:55:04 d2.evaluation.evaluator]: Inference done 30/559. Dataloading: 0.0009 s/iter. Inference: 0.2706 s/iter. Eval: 0.0001 s/iter. Total: 0.2717 s/iter. ETA=0:02:23
[10/13 22:55:09 d2.evaluation.evaluator]: Inference done 49/559. Dataloading: 0.0009 s/iter. Inference: 0.2721 s/iter. Eval: 0.0001 s/iter. Total: 0.2733 s/iter. ETA=0:02:19
[10/13 22:55:14 d2.evaluation.evaluator]: Inference done 67/559. Dataloading: 0.0009 s/iter. Inference: 0.2741 s/iter. Eval: 0.0001 s/iter. Total: 0.2753 s/iter. ETA=0:02:15
[10/13 22:55:20 d2.evaluation.evaluator]: Inference done 86/559. Dataloading: 0.0009 s/iter. Inference: 0.2748 s/iter. Eval: 0.0001 s/iter. Total: 0.2759 s/iter. ETA=0:02:10
[10/13 22:55:25 d2.evaluation.evaluator]: Inference done 104/559. Dataloading: 0.0009 s/iter. Inference: 0.2754 s/iter. Eval: 0.00

In [11]:
torch.save(cfg,'cfg.pt')
torch.save(cfg,'cfg.pth')
torch.save(predictor,'predictor.pt')
torch.save(predictor,'predictor.pth')
torch.save(evaluator,'evaluator.pt')
torch.save(evaluator,'evaluator.pth')
torch.save(v,'img.pt')
torch.save(v,'img.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')

In [12]:
torch.save(load_data(),'data.pt')
torch.save(load_data(),'data.pth')

100%|████████████████████████████████████████| 559/559 [00:01<00:00, 344.58it/s]
